In [53]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import json
import math

In [54]:
# Load rotation vectors for eavh camera
rvecs = []
for cam in range(1, 8):
    with open(f'cam_{cam}/extrinsics/rvec.txt', 'r') as f:
        rvecs.append(np.array([float(x) for x in f.read().split()]))
print(rvecs[0])

[ 2.05018189  0.98840169 -0.57373535]


In [55]:
# Convert rotation vectors to rotation matrices 
rmat = [cv.Rodrigues(rvec)[0] for rvec in rvecs]
# Transpose rotation matrices
rmat = [np.transpose(r) for r in rmat]

In [56]:
# Load translation vectors for each camera
tvecs = []
for cam in range(1, 8):
    with open(f'cam_{cam}/extrinsics/tvec.txt', 'r') as f:
        tvecs.append(np.array([float(x) for x in f.read().split()]))
print(tvecs[0])

[-134.25953329  -23.76099666  187.46374006]


In [57]:
# Construct camera to world transformation matrices
T = [np.eye(4) for _ in range(7)]
for i in range(7):
    T[i][:3, :3] = rmat[i]
    T[i][:3, 3] = tvecs[i]
print(T[0])

[[ 5.96808242e-01  4.51155708e-01 -6.63534814e-01 -1.34259533e+02]
 [ 7.99931863e-01 -3.99142022e-01  4.48101172e-01 -2.37609967e+01]
 [-6.26812263e-02 -7.98213113e-01 -5.99105074e-01  1.87463740e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [58]:
# Load intrinsic matrices for each camera
K = []
for cam in range(1, 8):
    with open(f'cam_{cam}/intrinsics/cameraMatrix.txt', 'r') as f:
        K.append(np.array([float(x) for x in f.read().split()]).reshape(3, 3))
print(K[0])

[[1.08437970e+03 0.00000000e+00 8.55671508e+02]
 [0.00000000e+00 1.06331702e+03 5.57359735e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [59]:
# Construct transforms.json in instant-ngp format
transforms = {}
transforms['w'] = 1920.0
transforms['h'] = 1080.0
transforms['aabb_scale'] = 2.0
transforms['scale'] = 0.005
transforms['offset'] = [0.0, 0.0, 0.0]
frames = []
for i in range(7):
    frame = {}
    frame['file_path'] = f'c{i+1}origalpha.png'
    frame['transform_matrix'] = T[i].tolist()
    frame['f1_x'] = K[i][0, 0]
    frame['f1_y'] = K[i][1, 1]
    frame['camera_angle_x'] = 2 * math.atan(1080 / (2 * K[i][0, 0]))
    frame['camera_angle_y'] = 2 * math.atan(1920 / (2 * K[i][1, 1]))
    frame['c1_x'] = K[i][0, 2]
    frame['c1_y'] = K[i][1, 2]
    frames.append(frame)
transforms['frames'] = frames

In [60]:
# Write transforms to file
with open('transforms.json', 'w') as f:
    json.dump(transforms, f, indent=4)